In [1]:
## WRF Test Exercise

In [ ]:
### Set-up case Study: Model Domain (1)


Main WRF page
	http://www2.mmm.ucar.edu/wrf/OnLineTutorial/CASES/JAN00/index.html


In ~/WRF/WPS directory. Edit namelist.wps 
	emacs namelist.wps 

Change to following (mostly getting rid of 2nd number and a ,)
```
	max_dom = 1
	parent_id = 1, 
    parent_grid_ratio = 1, 
    i_parent_start = 1, 
    j_parent_start = 1, 
    e_we = 74, 
    e_sn = 61, 
    geog_data_res = 'default',
```

Explanations of all variables are here
	http://www2.mmm.ucar.edu/wrf/users/docs/user_guide_V3/users_guide_chap3.htm
